# ===========================================
# 2_data_preprocessing.ipynb
# Bereinigung, Tokenisierung & Vektorisierung (Embeddings) für AskMeNow
# ===========================================

# Zelle 1: Bibliotheken importieren
import os
import re
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import nltk
from nltk.tokenize import word_tokenize
from tqdm import tqdm

# Stelle sicher, dass NLTK-Daten verfügbar sind
nltk.download('punkt')
%matplotlib inline

# Zelle 2: Pfade definieren und Rohdaten laden
RAW_DATA_DIR = os.path.join("..", "data", "raw")
PROCESSED_DATA_DIR = os.path.join("..", "data", "processed")
EMBEDDING_DIR = os.path.join("..", "data", "embeddings")
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
os.makedirs(EMBEDDING_DIR, exist_ok=True)

# Angenommen, die FAQ-Daten liegen in einer CSV-Datei "faq_data.csv" mit den Spalten "question" und "answer"
data_file = os.path.join(RAW_DATA_DIR, "faq_data.csv")
df = pd.read_csv(data_file)
print("Anzahl der FAQ-Einträge:", len(df))
display(df.head())

# Zelle 3: Datenbereinigung
def clean_text(text):
    """
    Entfernt HTML-Tags, überflüssige Leerzeichen und konvertiert den Text in Kleinbuchstaben.
    """
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip().lower()

df["clean_question"] = df["question"].apply(clean_text)
df["clean_answer"] = df["answer"].apply(clean_text)
print("\nBereinigte Fragen und Antworten:")
display(df[["clean_question", "clean_answer"]].head())

# Zelle 4: Optional: Tokenisierung (für weitere Analysen)
df["question_tokens"] = df["clean_question"].apply(word_tokenize)
df["answer_tokens"] = df["clean_answer"].apply(word_tokenize)
print("\nBeispiel tokenisierter Texte:")
display(df[["question_tokens", "answer_tokens"]].head())

# Zelle 5: Vektorisierung mit einem vortrainierten Transformer-Modell
# Wir nutzen z. B. "sentence-transformers/all-MiniLM-L6-v2" für die Berechnung von Satz-Embeddings.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Verwende Gerät:", device)

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model.to(device)
model.eval()

def get_sentence_embedding(text, tokenizer, model, device):
    """
    Berechnet das Embedding eines Satzes mittels Mean-Pooling der Token-Embeddings.
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state  # [batch_size, seq_len, hidden_dim]
    attention_mask = inputs["attention_mask"]
    mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
    masked_embeddings = embeddings * mask
    summed = torch.sum(masked_embeddings, dim=1)
    counts = torch.clamp(mask.sum(dim=1), min=1e-9)
    mean_pooled = summed / counts
    return mean_pooled.squeeze(0).cpu().numpy()

# Berechne Embeddings für alle Fragen und Antworten
question_embeddings = []
answer_embeddings = []

print("\nBerechne Embeddings für Fragen:")
for text in tqdm(df["clean_question"], desc="Fragen Embeddings"):
    emb = get_sentence_embedding(text, tokenizer, model, device)
    question_embeddings.append(emb)

print("\nBerechne Embeddings für Antworten:")
for text in tqdm(df["clean_answer"], desc="Antworten Embeddings"):
    emb = get_sentence_embedding(text, tokenizer, model, device)
    answer_embeddings.append(emb)

# Zelle 6: Speichern der Embeddings und der verarbeiteten Daten
# Speichere die Embeddings als NumPy-Arrays
np.save(os.path.join(EMBEDDING_DIR, "question_embeddings.npy"), np.array(question_embeddings))
np.save(os.path.join(EMBEDDING_DIR, "answer_embeddings.npy"), np.array(answer_embeddings))
print("\nEmbeddings wurden im Ordner", EMBEDDING_DIR, "gespeichert.")

# Speichere auch den bereinigten DataFrame als CSV
processed_csv_path = os.path.join(PROCESSED_DATA_DIR, "faq_processed.csv")
df.to_csv(processed_csv_path, index=False)
print("Verarbeitete FAQ-Daten gespeichert unter:", processed_csv_path)
